<a href="https://colab.research.google.com/github/karthikeyan19/AI-Music-LSTM/blob/master/MusicLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install music21

    100% |████████████████████████████████| 18.0MB 2.0MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /root/.cache/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21


In [0]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM,Bidirectional
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [61]:
  """ Get all the notes and chords from the midi files in the ./midi_songs directory """
  notes = []

  for file in glob.glob("*.mid"):
      midi = converter.parse(file)

      print("Parsing %s" % file)

      notes_to_parse = None

      try: # file has instrument parts
          s2 = instrument.partitionByInstrument(midi)
          notes_to_parse = s2.parts[0].recurse() 
      except: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes

      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))

  with open('data/notes', 'wb') as filepath:
      pickle.dump(notes, filepath)




Parsing cosmo.mid
Parsing FF4.mid
Parsing Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing ff8-lfp.mid
Parsing Still_Alive-1.mid
Parsing roseofmay-piano.mid
Parsing HighwindTakestotheSkies.mid
Parsing Final_Fantasy_Matouyas_Cave_Piano.mid
Parsing ff7-mainmidi.mid
Parsing thenightmarebegins.mid
Parsing FFIXQuMarshP.mid
Parsing tifap.mid
Parsing Eternal_Harvest.mid
Parsing 8.mid
Parsing ff11_awakening_piano.mid
Parsing Kingdom_Hearts_Dearly_Beloved.mid
Parsing rufus.mid
Parsing Life_Stream.mid
Parsing lurk_in_dark.mid
Parsing relmstheme-piano.mid
Parsing Cids.mid
Parsing dayafter.mid
Parsing Fyw_piano.mid
Parsing sandy.mid
Parsing EyesOnMePiano.mid
Parsing ultimafro.mid
Parsing Oppressed.mid
Parsing dontbeafraid.mid
Parsing gerudo.mid
Parsing AT.mid
Parsing FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing path_of_repentance.mid
Parsing Finalfantasy6fanfarecomplete.mid
Parsing DOS.mid
Parsing sera_.mid
Parsing Rydia_pc.mid
Parsing great_war.mid
Parsing ff4_piano_collections-m

FileNotFoundError: ignored

In [57]:
""" Prepare the sequences used by the Neural Network """
n_vocab = len(set(notes))

sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in notes))

 # create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)

network_output = np_utils.to_categorical(network_output)

TypeError: ignored

In [0]:
""" create the structure of the neural network """
import keras
model = Sequential()
model.add(Bidirectional(LSTM(
    512,
    return_sequences=True
),input_shape=(network_input.shape[1], network_input.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(512))
#odel.add(Dropout(0.3))
#odel.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0))


In [49]:
""" train the neural network """
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=100, batch_size=1024, callbacks=callbacks_list)


Epoch 1/100
57077/57077 [==============================] - 170s 3ms/step - loss: 4.3649
Epoch 2/100
57077/57077 [==============================] - 168s 3ms/step - loss: 4.2895
Epoch 3/100
57077/57077 [==============================] - 168s 3ms/step - loss: 4.2167
Epoch 4/100
57077/57077 [==============================] - 169s 3ms/step - loss: 4.1265
Epoch 5/100
57077/57077 [==============================] - 169s 3ms/step - loss: 4.0593
Epoch 6/100
57077/57077 [==============================] - 169s 3ms/step - loss: 3.9800
Epoch 7/100
57077/57077 [==============================] - 169s 3ms/step - loss: 3.9001
Epoch 8/100
57077/57077 [==============================] - 170s 3ms/step - loss: 3.8164
Epoch 9/100
57077/57077 [==============================] - 170s 3ms/step - loss: 3.7286
Epoch 10/100
57077/57077 [==============================] - 170s 3ms/step - loss: 3.6333
Epoch 11/100
57077/57077 [==============================] - 169s 3ms/step - loss: 3.5465
Epoch 12/100
57077/57077 [====

In [1]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

sequence_length = 100
network_input = []
output = []
print(notes)
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
normalized_input = normalized_input / float(n_vocab)
print(len(network_input))
start = numpy.random.randint(0, len(network_input)-1)

pitchnames = sorted(set(item for item in notes))
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []
temperature = 1.0

# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)
    prediction = prediction / temperature
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]

NameError: ignored

In [63]:
""" convert the output from the prediction to notes and create a midi file
        from the notes """
from music21 import instrument, note, stream, chord
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_output5.mid')

'test_output5.mid'